In [ ]:
import torch
import torchvision

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pathlib

# PyTorch - My Getting Started Notebook

I'm mainly following this tutorial: https://nextjournal.com/gkoehler/pytorch-mnist


Book about pytorch: https://pytorch.org/assets/deep-learning/Deep-Learning-with-PyTorch.pdf

Other useful resource: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

Also I'm using a net from a pytorch example: https://github.com/pytorch/examples/blob/master/mnist/main.py

In [ ]:
# HyperHyper!
n_epochs = 3

batch_size_train = 64
batch_size_test = 1000

learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
# torch.backends.cudnn.enabled = False # I don't have a nvidia gfx card on my laptop
torch.manual_seed(random_seed)

# Directory setup
result_path = pathlib.Path.cwd() / "results"
data_dir = pathlib.Path.cwd() / "data"

# Just checking if they already exist,
# if not create it.
print(result_path)
if not result_path.exists():
    print("Path not found...mkdir")
    result_path.mkdir()
else:
    print("Already exists")

print(data_dir)
if not data_dir.exists():
    print("Path not found...mkdir")
    data_dir.mkdir()
else:
    print("Already exists")


# Load the data

In [ ]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(data_dir, train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST(data_dir, train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

  # If he needs to download it, cause it's not already in the data folder, he/she/it would do so and say so below.

In [ ]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [ ]:
print(f"type: {type(example_data)}")
example_data.shape

In [ ]:
def plot_batch(batch, b_range=5):
    if isinstance(b_range, tuple):
        if len(b_range) ==2:
            (start, stop) = b_range
            step = 1
        elif len(b_range) ==3:
            (start, stop, step) = b_range
        else:
            raise ValueError("b_range Should have len==2")
    else:
        start, stop = 0, b_range
    fig = plt.figure()
    num_to_plot = int(np.ceil((stop - start) / step))
    print(f"num_to_plot: {num_to_plot}")
    cols = 4
    rows = int(np.ceil(num_to_plot / cols))
    print(f"rows, cols = {rows}, {cols}")
    for i in range(start, stop, step):
        subplot_num = int(1 + ((i - start)/step) % num_to_plot)
        print(f"i: {i} --> subplot#: {subplot_num}")
        plt.subplot(rows, cols, subplot_num)
        plt.imshow(batch[i][0], cmap='gray', interpolation='none')
        plt.title(f"Label(#{i}): {example_targets[i]}")
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout()
    fig

In [ ]:
plot_batch(example_data, (5,20, 2))

# Define our model

see `petenet.py`

In [ ]:
# Some stuff we need for models
from ZeNet.nets import Net1, Net2
import torch.optim as optim
import torch.nn as nn # contain trainable params
import torch.nn.functional as F # purely functional

## Initialize the model

In [ ]:
network = Net1()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

# If we want to use another device then we'd have to/should sent the network parameters to the device, e.g. for a CUDA GPU `network.cuda`

In [ ]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), result_path/'model.pth')
      torch.save(optimizer.state_dict(), result_path/'optimizer.pth')

def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))
  
def train_and_test():
  train_losses = []
  train_counter = []
  test_losses = []
  test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

  test()
  for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()
  return train_losses, train_counter, test_losses, test_counter

In [ ]:

train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()


In [ ]:
train_losses1 = train_losses
train_counter1 = train_counter
test_losses1 = test_losses
test_counter1 = test_counter

network1 = network

In [ ]:
network = Net1()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses2, train_counter2, test_losses2, test_counter2 = train_and_test()
network2 = network